In [1]:
#各ライブラリのimport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#欠損値補完候補探索
#car_name = ['amc concord dl' , 'renault 18i' , 'ford pinto' , 'ford maverick' , 'ford mustang cobra']
#df_train_test[df_train_test['car name'] == car_name[3]]
#ford pintoは5台存在
#ford maverickは3台存在
#displacement =[151 , 100 , 85 , 140]
#df_train_test[df_train_test['displacement'] == displacement[0]]
#df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]


In [43]:
#データ読み込みから標準化まで(train,test結合版)
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('../train.tsv' , sep = '\t')
df_test = pd.read_csv('../test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
#print(df_train_test[df_train_test['horsepower'] == "?"])
df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train_test[df_train_test['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train_test.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train_test.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train_test.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train_test.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)
#ford maverick
df_train_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean(numeric_only=True)
#ford mustang cobra
df_train_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean(numeric_only=True)
#補完確認
df_train_test[df_train_test['horsepower'].isnull()]
#ラベルエンコーディング(train,test結合版)
#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({'chevroelt':'chevrolet' , 'toyouta':'toyota'})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train_test['manufacturers name'].values.reshape(-1,1)) , columns = df_train_test['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train_test['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_number = pd.merge(df_manufacturers_number , df_origin_number , how = 'outer' , right_index = True , left_index=True)
#df_numberとdf_train_testの結合
#右側のデータフレームのインデックスをキーとして、左側のデータフレームを結合する(right_index=True)
#右側のデータフレームのインデックスをキーとして、左側のデータフレームを結合する(right_index=True)
#left_indexとright_indexの両方をTrueにしないとエラーになってしまった
df_train = pd.merge(df_train_test , df_number , how = 'outer' , right_index = True , left_index=True)
df_train = df_train.dropna(how = 'any')
print(df_train.columns)
df_test = pd.merge(df_train_test , df_number , how = 'outer' , right_index = True , left_index=True)
df_test = df_test.dropna(how = 'any')
#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'car name' , 'manufacturers name' , 'origin'] , axis = 1)
df_test_drop = df_test.drop(['id' , 'car name' , 'manufacturers name' , 'origin'] , axis = 1)
scaler.fit(df_train_drop)
df_train_std= pd.DataFrame(scaler.transform(df_train_drop), columns = df_train_drop.columns)
df_test_std = pd.DataFrame(scaler.transform(df_train_drop), columns = df_train_drop.columns)



Index(['id', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name',
       'manufacturers name', 'dodge', 'vw', 'toyota', 'pontiac', 'chevrolet',
       'oldsmobile', 'bmw', 'mercedes-benz', 'datsun', 'amc', 'renault',
       'peugeot', 'ford', 'mercury', 'subaru', 'honda', 'volkswagen', 'saab',
       'mazda', 'plymouth', 'opel', 'chevy', 'capri', 'fiat', 'hi', 'audi',
       'buick', 'vokswagen', 'volvo', 'triumph', 'America', 'Europe', 'Japan'],
      dtype='object')


c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
#読み込みから標準化まで(trainのみ)
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('../train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)

#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({'chevroelt':'chevrolet' , 'toyouta':'toyota'})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_train_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train['manufacturers name'].values.reshape(-1,1)) , columns = df_train['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_train_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_train_number = pd.merge(df_train_manufacturers_number , df_train_origin_number , how = 'outer' , right_index = True , left_index=True)
#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'manufacturers name' , 'car name', 'origin'] , axis = 1)
print(df_train_drop.info())
df_train_std = pd.DataFrame(scaler.fit_transform(df_train_drop), columns = df_train_drop.columns)

df_train = pd.merge(df_train_std , df_train_number , how = 'outer' , right_index = True , left_index=True)
df_train = df_train.dropna(how = 'any')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           199 non-null    float64
 1   cylinders     199 non-null    int64  
 2   displacement  199 non-null    float64
 3   horsepower    199 non-null    float64
 4   weight        199 non-null    float64
 5   acceleration  199 non-null    float64
 6   model year    199 non-null    int64  
dtypes: float64(5), int64(2)
memory usage: 11.0 KB
None


c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
#データ読み込み
df_train = pd.read_csv('../train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

,id,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
24,48,23.0,4,151.0,NaN,3035.0,20.5,82,1,amc concord dl
113,234,34.5,4,100.0,NaN,2320.0,15.8,81,2,renault 18i
145,288,40.9,4,85.0,NaN,1835.0,17.3,80,2,renault lecar deluxe
175,349,25.0,4,98.0,NaN,2046.0,19.0,71,1,ford pinto


In [35]:
#trainデータを用いて実装
#ランダムフォレスト(weight,cylinder(排気量と相関が強いカラム)抜きが一番精度高かった)
#読み込みから標準化まで(trainのみ)
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('../train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)

#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({'chevroelt':'chevrolet' , 'toyouta':'toyota'})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_train_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train['manufacturers name'].values.reshape(-1,1)) , columns = df_train['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_train_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_train_number = pd.merge(df_train_manufacturers_number , df_train_origin_number , how = 'outer' , right_index = True , left_index=True)


df_train = pd.merge(df_train , df_train_number , how = 'outer' , right_index = True , left_index=True)
df_train = df_train.dropna(how = 'any')
print(df_train.columns)

#目的変数の分離
features = df_train.drop(['mpg' , 'manufacturers name' , 'car name' , 'id'  ,'cylinders' , 'origin' , 'weight'] , axis = 1)
target = df_train['mpg']
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.3 , random_state = 0)

#学習
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100 , random_state = 0)
model.fit(features_train , target_train)
#予測
pred = model.predict(features_test)
#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(target_test, pred)) 

Index(['id', 'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name',
       'manufacturers name', 'dodge', 'vw', 'toyota', 'pontiac', 'chevrolet',
       'oldsmobile', 'bmw', 'mercedes-benz', 'datsun', 'amc', 'renault',
       'peugeot', 'ford', 'mercury', 'subaru', 'honda', 'volkswagen', 'saab',
       'mazda', 'plymouth', 'opel', 'chevy', 'capri', 'fiat', 'hi', 'audi',
       'buick', 'vokswagen', 'volvo', 'triumph', 'America', 'Europe', 'Japan'],
      dtype='object')


c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


2.6003598885538883

In [38]:
#重回帰分析
#読み込みから標準化まで(trainのみ)
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('../train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)

#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({'chevroelt':'chevrolet' , 'toyouta':'toyota'})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_train_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train['manufacturers name'].values.reshape(-1,1)) , columns = df_train['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_train_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_train_number = pd.merge(df_train_manufacturers_number , df_train_origin_number , how = 'outer' , right_index = True , left_index=True)
#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'manufacturers name' , 'car name', 'origin'] , axis = 1)
df_train_std = pd.DataFrame(scaler.fit_transform(df_train_drop), columns = df_train_drop.columns)

df_train = pd.merge(df_train_std , df_train_number , how = 'outer' , right_index = True , left_index=True)
df_train = df_train.dropna(how = 'any')

#特徴量の分割
features = df_train_std.drop(['mpg','cylinders' , 'weight'] , axis = 1)
target = df_train_std['mpg']
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.25 , random_state = 0)
#ライブラリのimport
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#学習と評価
model.fit(features_train , target_train)
pred = model.predict(features_test)
#評価
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(target_test, pred)) 

c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.4029484468959302

In [ ]:
#ニューラルネットワーク
#目的変数の分離
features = df_train.drop(['id' , 'mpg' , 'horsepower' ,'weight' , 'car name' , 'acceleration', 'origin'] , axis = 1)
target = df_train['mpg']
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , traget_test = train_test_split(features , target , test_size = 0.3 , random_state = 0)

#学習と評価
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes =100 , max_iter=200)
model.fit(features_train , target_train)
pred = model.predict(features_test)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(target_test , pred)

In [42]:
#XGBoost
#読み込みから標準化まで(trainのみ)
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('../train.tsv' , sep = '\t')
df_train['horsepower'] = df_train['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出

#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train[df_train['displacement'] == 151]['horsepower'].mean(numeric_only=True)
#renault 18
df_train.iloc[113 , 4] = df_train[(98 <= df_train['displacement']) & (df_train['displacement']<= 102)]['horsepower'].mean(numeric_only=True)
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train[df_train['displacement'] == 85]['horsepower'].mean(numeric_only=True)
#ford pinto
df_train.iloc[175 , 4] = df_train[df_train['car name'] == 'ford pinto']['horsepower'].mean(numeric_only=True)

#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train['manufacturers name'] = df_split.iloc[:, 0]
df_train['manufacturers name'] = df_train['manufacturers name'].replace({'chevroelt':'chevrolet' , 'toyouta':'toyota'})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_train_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train['manufacturers name'].values.reshape(-1,1)) , columns = df_train['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_train_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_train_number = pd.merge(df_train_manufacturers_number , df_train_origin_number , how = 'outer' , right_index = True , left_index=True)
#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'manufacturers name' , 'car name', 'origin'] , axis = 1)
df_train_std = pd.DataFrame(scaler.fit_transform(df_train_drop), columns = df_train_drop.columns)

df_train = pd.merge(df_train_std , df_train_number , how = 'outer' , right_index = True , left_index=True)
df_train = df_train.dropna(how = 'any')

#特徴量の分割
features = df_train_std.drop(['mpg','cylinders'] , axis = 1)
target = df_train_std['mpg']
#データ分割
from sklearn.model_selection import train_test_split
features_train , features_test , target_train , target_test = train_test_split(features , target , test_size = 0.25 , random_state = 0)
#学習と評価
from xgboost import XGBRegressor
#early_stopping_roundsは学習回数を適切なタイミングで打ち止めるための仕組み
model = XGBRegressor(n_estimators = 1000, early_stopping_rounds = 10)
model.fit(features_train , target_train)


c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: Must have at least 1 validation dataset for early stopping.